In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [48]:
df = pd.read_csv("/content/drive/MyDrive/TASK1_VERV/electricity-consumption/AEP_hourly.csv")
print("="*50)
print("First Five Rows ","\n")
print(df.head(5),"\n")
print(df.tail(5),"\n")
print("="*50)
print("Information About Dataset","\n")
print(df.info(),"\n")

print("="*50)
print("Describe the Dataset ","\n")
print(df.describe(),"\n")

print("="*50)
print("Null Values t ","\n")
print(df.isnull().sum(),"\n")

First Five Rows  

              Datetime   AEP_MW
0  2004-12-31 01:00:00  13478.0
1  2004-12-31 02:00:00  12865.0
2  2004-12-31 03:00:00  12577.0
3  2004-12-31 04:00:00  12517.0
4  2004-12-31 05:00:00  12670.0 

                   Datetime   AEP_MW
121268  2018-01-01 20:00:00  21089.0
121269  2018-01-01 21:00:00  20999.0
121270  2018-01-01 22:00:00  20820.0
121271  2018-01-01 23:00:00  20415.0
121272  2018-01-02 00:00:00  19993.0 

Information About Dataset 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121273 entries, 0 to 121272
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Datetime  121273 non-null  object 
 1   AEP_MW    121273 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.9+ MB
None 

Describe the Dataset  

              AEP_MW
count  121273.000000
mean    15499.513717
std      2591.399065
min      9581.000000
25%     13630.000000
50%     15310.000000
75%     17200.000000
max     25695.

In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/TASK1_VERV/electricity-consumption/AEP_hourly.csv")

# Assuming 'Megawatthours' is the target variable to predict
target_column = 'AEP_MW'

# Features (X) and target variable (y)
X = df.drop(target_column, axis=1)
y = df[target_column]

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [50]:
from tensorflow.keras.models import load_model

# Load the model
regressor = load_model("/content/drive/MyDrive/TASK1_VERV/electricity-consumption/models/my_lstm_model.h5")

In [69]:
def pre_pross_data_day (df):
  # Extract all Data Like Year MOnth Day Time etc
  dataset = df
  dataset["Month"] = pd.to_datetime(df["Datetime"]).dt.month
  dataset["Year"] = pd.to_datetime(df["Datetime"]).dt.year
  dataset["Date"] = pd.to_datetime(df["Datetime"]).dt.date
  dataset["Time"] = pd.to_datetime(df["Datetime"]).dt.time
  dataset["Week"] = pd.to_datetime(df["Datetime"]).dt.week
  dataset["Day"] = pd.to_datetime(df["Datetime"]).dt.day_name()
  dataset = df.set_index("Datetime")
  dataset.index = pd.to_datetime(dataset.index)
  ##########
  NewDataSet = dataset.resample('D').mean()
  #################
  print("Old Dataset ",dataset.shape )
  print("New  Dataset ",NewDataSet.shape )
  #####spliting dta
  TestData = NewDataSet.tail(100)

  Training_Set = NewDataSet.iloc[:,0:1]

  Training_Set = Training_Set[:-60]
  ##
  Training_Set = Training_Set
  sc = MinMaxScaler(feature_range=(0, 1))
  Train = sc.fit_transform(Training_Set)
  ##### ather prepration
  X_Train = []
  Y_Train = []

  # Range should be fromm 60 Values to END
  for i in range(60, Train.shape[0]):

      # X_Train 0-59
      X_Train.append(Train[i-60:i])

      # Y Would be 60 th Value based on past 60 Values
      Y_Train.append(Train[i])

  # Convert into Numpy Array
  X_Train = np.array(X_Train)
  Y_Train = np.array(Y_Train)

  print("X_Train.shape =",X_Train.shape)
  print("Y_Train.shape =",Y_Train.shape)
  # Shape should be Number of [Datapoints , Steps , 1 )
  # we convert into 3-d Vector or #rd Dimesnsion
  X_Train = np.reshape(X_Train, newshape=(X_Train.shape[0], X_Train.shape[1], 1))
  ##########
  Df_Total = pd.concat((NewDataSet[["AEP_MW"]], TestData[["AEP_MW"]]), axis=0)
  inputs = Df_Total[len(Df_Total) - len(TestData) - 60:].values

  # We need to Reshape
  inputs = inputs.reshape(-1,1)

  # Normalize the Dataset
  inputs = sc.transform(inputs)

  X_test = []
  for i in range(60, 160):
      X_test.append(inputs[i-60:i])

  # Convert into Numpy Array
  X_test = np.array(X_test)

  # Reshape before Passing to Network
  X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
  y_test = TestData["AEP_MW"].to_list()
  '''
  ###########################################################################
  predicted_stock_price = regressor.predict(X_test)
  predicted_stock_price = sc.inverse_transform(predicted_stock_price)

  True_MegaWatt = TestData["AEP_MW"].to_list()

  Predicted_MegaWatt  = predicted_stock_price
  dates = TestData.index.to_list()

  Machine_Df = pd.DataFrame(data={
    "Date":dates,
    "TrueMegaWatt": True_MegaWatt,
    "PredictedMeagWatt":[x[0] for x in Predicted_MegaWatt ]
  })
  fig = plt.figure()

  ax1= fig.add_subplot(111)

  x = dates
  y = True_MegaWatt

  y1 = Predicted_MegaWatt

  plt.plot(x,y, color="green")
  plt.plot(x,y1, color="red")
  # beautify the x-labels
  plt.gcf().autofmt_xdate()
  plt.xlabel('Dates')
  plt.ylabel("Power in MW")
  plt.title("Machine Learned the Pattern Predicting Future Values ")
  plt.legend()

  return Machine_Df'''

  ###########################################################################
  return X_Train,y_train,X_test,y_test,sc


In [74]:
def predict_lstm(regressor,X_test,y_test,sc) :
  True_MegaWatt = y_test
  dates = X_test.index.to_list()
  X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
  y_test = y_test.to_list()
  predicted_stock_price = regressor.predict(X_test)
  predicted_stock_price = sc.inverse_transform(predicted_stock_price)

  Predicted_MegaWatt  = predicted_stock_price

  Machine_Df = pd.DataFrame(data={
    "Date":dates,
    "TrueMegaWatt": True_MegaWatt,
    "PredictedMeagWatt":[x[0] for x in Predicted_MegaWatt ]
  })
  fig = plt.figure()

  ax1= fig.add_subplot(111)

  x = dates
  y = True_MegaWatt

  y1 = Predicted_MegaWatt

  plt.plot(x,y, color="green")
  plt.plot(x,y1, color="red")
  # beautify the x-labels
  plt.gcf().autofmt_xdate()
  plt.xlabel('Dates')
  plt.ylabel("Power in MW")
  plt.title("Machine Learned the Pattern Predicting Future Values ")
  plt.legend()

  return Machine_Df


In [75]:
x_train,ytrain,x_text,y_test,sc=pre_pross_data_day(df)

Old Dataset  (121273, 7)
New  Dataset  (5055, 4)
X_Train.shape = (4935, 60, 1)
Y_Train.shape = (4935, 1)


<ipython-input-69-8dbbcb3ca23f>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  dataset["Week"] = pd.to_datetime(df["Datetime"]).dt.week
<ipython-input-69-8dbbcb3ca23f>:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  NewDataSet = dataset.resample('D').mean()
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [76]:
y_train.shape

(97018,)

In [77]:
result_pre=predict_lstm(regressor,X_test,y_test,sc)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

### TIP 2


In [63]:
import pandas as pd

def read_file(file_path):
    """
    Read the CSV file and return the DataFrame.

    Parameters:
    - file_path: Path to the CSV file

    Returns:
    - df: DataFrame containing the data
    """
    df = pd.read_csv(file_path)
    return df


In [86]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def preprocess_data(df):
    """
    Preprocess the data for LSTM model training and prediction.

    Parameters:
    - df: DataFrame containing the data

    Returns:
    - X_train: Processed training features
    - y_train: Processed training labels
    - X_test: Processed testing features
    - y_test: Testing labels
    """
    # Reformat Date Time Columns
    dataset = df
    dataset["Datetime"] = pd.to_datetime(df["Datetime"])
    dataset["Month"] = dataset["Datetime"].dt.month
    dataset["Year"] = dataset["Datetime"].dt.year
    dataset["Date"] = dataset["Datetime"].dt.date
    dataset["Time"] = dataset["Datetime"].dt.time
    dataset["Week"] = dataset["Datetime"].dt.week
    dataset["Day"] = dataset["Datetime"].dt.day_name()
    dataset = dataset.set_index("Datetime")

    # Resample data to daily frequency
    NewDataSet = dataset.resample('D').sum()

    # Splitting data
    TestData = NewDataSet.tail(100)
    Training_Set = NewDataSet.iloc[:, 0:1]
    Training_Set = Training_Set[:-60]

    # Normalization
    sc = MinMaxScaler(feature_range=(0, 1))
    Train = sc.fit_transform(Training_Set)

    X_Train = []
    Y_Train = []

    for i in range(60, Train.shape[0]):
        X_Train.append(Train[i-60:i])
        Y_Train.append(Train[i])

    X_Train = np.array(X_Train)
    Y_Train = np.array(Y_Train)
    X_Train = np.reshape(X_Train, newshape=(X_Train.shape[0], X_Train.shape[1], 1))

    Df_Total = pd.concat((NewDataSet[["AEP_MW"]], TestData[["AEP_MW"]]), axis=0)
    inputs = Df_Total[len(Df_Total) - len(TestData) - 60:].values
    inputs = inputs.reshape(-1, 1)
    inputs = sc.transform(inputs)

    X_test = []
    for i in range(60, 160):
        X_test.append(inputs[i-60:i])

    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    y_test = TestData["AEP_MW"].to_list()

    return X_Train, Y_Train, X_test, y_test


In [87]:
def predict_lstm2(regressor, X_test, y_test, scaler):
    """
    Make predictions using the trained LSTM model.

    Parameters:
    - regressor: Trained LSTM model
    - X_test: Testing features
    - y_test: True labels for testing
    - scaler: Scaler object for inverse transforming predictions

    Returns:
    - Machine_Df: DataFrame with True and Predicted values
    - mse: Mean Squared Error
    - mae: Mean Absolute Error
    """
    predicted_stock_price = regressor.predict(X_test)
    predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

    True_MegaWatt = y_test
    Predicted_MegaWatt = [x[0] for x in predicted_stock_price]
    dates = pd.date_range(start="2023-06-09", periods=len(True_MegaWatt), freq='D')

    Machine_Df = pd.DataFrame(data={
        "Date": dates,
        "TrueMegaWatt": True_MegaWatt,
        "PredictedMeagWatt": Predicted_MegaWatt
    })

    # Convert lists to numpy arrays
    True_MegaWatt = np.array(True_MegaWatt)
    Predicted_MegaWatt = np.array(Predicted_MegaWatt)

    mse = np.mean((True_MegaWatt - Predicted_MegaWatt) ** 2)
    mae = np.mean(np.abs(True_MegaWatt - Predicted_MegaWatt))

    return Machine_Df, mse, mae


In [88]:
# Read the file
file_path = "/content/drive/MyDrive/TASK1_VERV/electricity-consumption/AEP_hourly.csv"
df = read_file(file_path)

# Preprocess the data
X_train, Y_train, X_test, y_test = preprocess_data(df)

# Train the LSTM model (you already have this code)

# Make predictions
Machine_Df, mse, mae = predict_lstm2(regressor, X_test, y_test, sc)

# Plot and display results (you already have this code)


1/4 [======>.......................] - ETA: 0s

<ipython-input-86-b3900d1db2d6>:25: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  dataset["Week"] = dataset["Datetime"].dt.week
<ipython-input-86-b3900d1db2d6>:30: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  NewDataSet = dataset.resample('D').sum()
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


4/4 [==============================] - 0s 18ms/step


In [89]:
Machine_Df

,Date,TrueMegaWatt,PredictedMeagWatt
0,2023-06-09,315787.0,9387.287109
1,2023-06-10,311136.0,14127.167969
2,2023-06-11,293702.0,15021.509766
3,2023-06-12,291763.0,14313.794922
4,2023-06-13,322644.0,12582.367188
...,...,...,...
95,2023-09-12,368834.0,15160.231445
96,2023-09-13,364327.0,15840.702148
97,2023-09-14,363628.0,15107.758789
98,2023-09-15,376504.0,15169.905273


## TIP 3


In [93]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt


In [100]:
def train_and_predict_lstm(regressor, X_train, y_train, scaler):
    """
    Train the LSTM model and make predictions on the training data.

    Parameters:
    - regressor: LSTM model
    - X_train: Training features
    - y_train: Training labels
    - scaler: Scaler object

    Returns:
    - Machine_Df: DataFrame with True and Predicted values
    - mse: Mean Squared Error
    - mae: Mean Absolute Error
    """
    # Train the LSTM model
    regressor.fit(X_train, y_train, epochs=50, batch_size=32)

    # Make predictions on the training data
    predicted_stock_price = regressor.predict(X_train)
    predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

    True_MegaWatt = y_train
    Predicted_MegaWatt = [x[0] for x in predicted_stock_price]
    dates = pd.date_range(start="2004-01-02", periods=len(True_MegaWatt), freq='D')

    Machine_Df = pd.DataFrame(data={
        "Date": dates,
        "TrueMegaWatt": True_MegaWatt,
        "PredictedMeagWatt": Predicted_MegaWatt
    })

    mse = np.mean((True_MegaWatt - Predicted_MegaWatt) ** 2)
    mae = np.mean(np.abs(True_MegaWatt - Predicted_MegaWatt))

    return Machine_Df, mse, mae


In [101]:
def preprocess_data_all(df):
    """
    Preprocess the raw dataframe for LSTM training.

    Parameters:
    - df: Raw dataframe with electricity consumption data

    Returns:
    - X_train: Preprocessed training features
    - y_train: Preprocessed training labels
    """
    # Extracting relevant features and creating new features
    df["Month"] = pd.to_datetime(df["Datetime"]).dt.month
    df["Year"] = pd.to_datetime(df["Datetime"]).dt.year
    df["Date"] = pd.to_datetime(df["Datetime"]).dt.date
    df["Time"] = pd.to_datetime(df["Datetime"]).dt.time
    df["Week"] = pd.to_datetime(df["Datetime"]).dt.isocalendar().week
    df["Day"] = pd.to_datetime(df["Datetime"]).dt.day_name()

    # Setting index and resampling by day
    dataset = df.set_index("Datetime")
    dataset.index = pd.to_datetime(dataset.index)
    NewDataSet = dataset.resample('D').mean()

    # Splitting into training and testing sets
    TestData = NewDataSet.tail(100)
    Training_Set = NewDataSet.iloc[:, 0:1]
    Training_Set = Training_Set[:-60]

    # Scaling the data
    sc = MinMaxScaler(feature_range=(0, 1))
    Train = sc.fit_transform(Training_Set)

    # Creating sequences for LSTM
    X_train = []
    y_train = []

    for i in range(60, Train.shape[0]):
        X_train.append(Train[i-60:i])
        y_train.append(Train[i])

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    X_train = np.reshape(X_train, newshape=(X_train.shape[0], X_train.shape[1], 1))

    return X_train, y_train,sc


In [102]:
def initialize_lstm_model():
    """
    Initialize the LSTM model.

    Returns:
    - regressor: Initialized LSTM model
    """
    regressor = Sequential()
    regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units=50))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units=32, activation="relu"))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units=1))

    regressor.compile(optimizer="adam", loss="mean_squared_error")

    return regressor


In [103]:
# Load the data
df = pd.read_csv("/content/drive/MyDrive/TASK1_VERV/electricity-consumption/AEP_hourly.csv")

# Preprocess the data
X_train, y_train,sc = preprocess_data_all(df)

# Initialize the model
regressor2 = initialize_lstm_model()

# Train the model and make predictions
Machine_Df, mse, mae = train_and_predict_lstm(regressor2, X_train, y_train, sc)

# Plot and display results
plt.plot(Machine_Df["Date"], Machine_Df["TrueMegaWatt"], color="green", label="True MegaWatt")
plt.plot(Machine_Df["Date"], Machine_Df["PredictedMeagWatt"], color="red", label="Predicted MegaWatt")
plt.xlabel("Date")
plt.ylabel("Power in MW")
plt.title("Machine Learned the Pattern Predicting Future Values")
plt.legend()
plt.show()

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

# Save the model
regressor.save("/content/drive/MyDrive/TASK1_VERV/electricity-consumption/models/my_lstm_model.h5")


<ipython-input-101-2ea703490255>:23: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  NewDataSet = dataset.resample('D').mean()


Epoch 1/50
155/155 [==============================] - 17s 73ms/step - loss: 0.0261
Epoch 2/50
155/155 [==============================] - 11s 73ms/step - loss: 0.0199
Epoch 3/50
155/155 [==============================] - 12s 75ms/step - loss: 0.0180
Epoch 4/50
155/155 [==============================] - 11s 73ms/step - loss: 0.0173
Epoch 5/50
155/155 [==============================] - 11s 73ms/step - loss: 0.0164
Epoch 6/50
155/155 [==============================] - 11s 73ms/step - loss: 0.0158
Epoch 7/50
155/155 [==============================] - 11s 73ms/step - loss: 0.0157
Epoch 8/50
155/155 [==============================] - 11s 72ms/step - loss: 0.0149
Epoch 9/50
155/155 [==============================] - 11s 72ms/step - loss: 0.0135
Epoch 10/50
155/155 [==============================] - 11s 73ms/step - loss: 0.0105
Epoch 11/50
155/155 [==============================] - 11s 72ms/step - loss: 0.0089
Epoch 12/50
155/155 [==============================] - 11s 73ms/step - loss: 0.0085
E

ValueError: Per-column arrays must each be 1-dimensional